In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
import sys, argparse, logging
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz

import pyximport

pyximport.install(setup_args={"include_dirs": np.get_include()})
from tmqrfeed.quotes.compress_daily_ohlcv import compress_daily
import os

from tmqrfeed.assetsession import AssetSession
from tmqrfeed.quotes.dataframegetter import DataFrameGetter
from tmqrfeed.datafeed import DataFeed
from tmqrfeed.manager import DataManager
from tmqrfeed.contracts import FutureContract, OptionContract
from tmqrfeed.quotes.quote_contfut import QuoteContFut
from tmqr.errors import *

In [ ]:
dm = DataManager()

In [ ]:
INSTRUMENT = "US.CL"

In [ ]:
dm.datafeed.get_instrument_info(INSTRUMENT)._info_dict

# Check the future contract selection

In [ ]:


"""
        Get future contract from futures chains
        :param instrument: Full-qualified instrument name
        :param date: current date 
        :param offset: future expiration offset, 0 - front month, +1 - front+1, etc. (default: 0)
        :return: Future contract
"""
dt = datetime(2011, 12, 1)
dm.chains_futures_get(INSTRUMENT, dt, offset=0)

### Full futures chains list (next 7 futures)

In [ ]:
pd.DataFrame(dm.datafeed.get_fut_chain(INSTRUMENT).get_list(dt), columns=['Contract', 'StartDate', 'EndDate']).head(7)

# Check the option contract selection

In [ ]:
# You can play with opt_offset=1, opt_min_days=2 parameters
"""
        Find future+option chain by given criteria
        :param instrument: Full-qualified instrument name
        :param date: current date
        :param kwargs: 
            - 'opt_offset' - option expiration offset, 0 - front month, +1 - front+1, etc. (default: 0)
            - 'opt_min_days' - minimal days count until option expiration (default: 2)
            - 'error_limit' - ChainNotFoundError error limit, useful to increase when you are trying to get far 'opt_offset' (default: 4)
        :return: (tuple) FutureContract, OptionChain
"""
dt = datetime(2011, 12, 1)
dm.chains_options_get(INSTRUMENT, dt, opt_offset=0, opt_min_days=2)

### Full options chains for every future contract

In [ ]:
futures_list = dm.datafeed.get_fut_chain(INSTRUMENT).get_list(dt, limit=4)
for f in futures_list:
    print(f"Future contract: {f[0]}")
    print("Options:")
    print(dm.datafeed.get_option_chains(f[0]))
    
    